# Create AWS instance

Mostly from http://chrisalbon.com/jupyter/run_project_jupyter_on_amazon_ec2.html, with a few tweaks

1. create AWS account
1. launch T2.micro ubuntu 16.04 instance
1. store .pem file
1. security group, Inbound rules, "AnyWhere"
    - HTTP, TCP, port 80
    - SSH, TCP, port 22
    - Custom TCP rule, TCP, port 8888 (for Jupyter)
    - Custom TCP rule, TCP, port 5901 (for vncserver)
1. create elastic ip and associate to instance above (if not, your IP will change each time you reboot)
1. download and install putty and puttygen
1. convert .pem to .ppk using puttygen.  Store both
1. ssh into the instance
    1. open Putty
    1. Use Elastic IP address, port 22, SSH
    1. Connection -> Data -> username = ubuntu
    1. Connection -> SSH -> Auth -> set private key to the .ppk file created above
    1. Save settings  
    1. save profile
    1. open
    1. click okay if rsa popup
1. Update system software
    - sudo apt-get update && sudo apt-get upgrade && sudo apt-get dist-upgrade && sudo apt-get autoremove && sudo apt clean && sudo reboot

# Vim quick start
- i - enters insert mode, esc enters command mode
- in command mode
    - :w - saves
    - :q - quits

# Install GUI (optional, but convenient)
1. First step takes about 10 minutes
    - sudo apt-get install ubuntu-desktop gnome-panel gnome-settings-daemon metacity nautilus gnome-terminal gnome-session dconf-editor tightvncserver
1. vncserver
1. vncserver -kill :1
1. cd /home/ubuntu/.vnc/
1. sudo rm xstartup
1. sudo vim xstartup
1. paste script below called "Script /home/ubuntu/.vnc/xstartup"
1. save & exit
1. sudo chmod +x xstartup
1. Make vncserver auto launch at startup
    1. Check if "upstart" is the init process
        - sudo stat /proc/1/exe
        - If you see "upstart", skip to next step.  But if you see "systemd", run the command below.
        - sudo apt-get install upstart-sysv && sudo update-initramfs -u && sudo reboot
    1. cd /etc/init
    1. sudo vim vncserver.conf
    1. paste script below called "Script /etc/init/vncserver.conf"
    1. save & exit
    1. Should now auto launch on boot.  To start/stop manually: sudo service vncserver start/stop
1. Install TightVNCviewer (or equivalent) on your local machine
1. Connect using AWS elastic IP with ":1" appended

# Install Python
1. Find linux anaconda installer url https://www.continuum.io/downloads
1. wget "that url"
1. bash "that file" - runs installer
    - accept /home/ubuntu/anaconda3
    - accept all prompts (specifically allow it to add to system path; if you miss it, do export PATH=~/anaconda3/bin:$PATH)
1. sudo rm "that file"
1. close terminal and open a new one
1. conda update anaconda
1. mkdir /home/ubuntu/notebooks
1. Follow rest of instructions at http://chrisalbon.com/jupyter/run_project_jupyter_on_amazon_ec2.html with following changes
    - Make sure you copy your password hash for later use.  It looks like 'sha1:98ff0e580111:12798c72623a6eecd54b51c006b1050f0ac1a62d'
    - While editing config file, add c.NotebookApp.notebook_dir = u"/home/ubuntu"
1. (optional) Install Jupyter Extensions from https://github.com/ipython-contrib/jupyter_contrib_nbextensions
1. Make Jupyer auto launch at startup
    1. Check if "upstart" is the init process
        - sudo stat /proc/1/exe
        - If you see "upstart", skip to next step.  But if you see "systemd", run the command below.
        - sudo apt-get install upstart-sysv && sudo update-initramfs -u && sudo reboot
    1. cd /etc/init
    1. sudo vim jupyter.conf
    1. paste script below called "Script /etc/init/jupyter.conf"
    1. save & exit
    1. Should now auto launch on boot.  To start/stop manually: sudo service jupyter start/stop

# Install Orange
1. conda create python=3 --name orange3
1. source activate orange3
1. conda config --add channels conda-forge
1. conda install orange3
1. cd home/ubuntu
1. sudo vim orange3.launcher
1. paste script below called "Script home/ubuntu/orange3.launcher"
1. save & exit
1. sudo chmod +x orange3.launcher
1. Launch desktop gui
1. System tools -> dconf editor -> org -> gnome -> nautilus -> preferences  (if no dconf-editor, do sudo apt-get install dconf-editor)
1. Executable-text-activation -> launch
1. Double click on orange3.launcher (may take a minute to load first time)

In [ ]:
# Clones Vanderplas books
! git clone https://github.com/jakevdp/WhirlwindTourOfPython.git /home/ubuntu/notebooks/jakevdp/WhirlwindTourOfPython
! git clone https://github.com/jakevdp/PythonDataScienceHandbook.git /home/ubuntu/notebooks/jakevdp/PythonDataScienceHandbook

# Setup rsa github access
https://help.github.com/articles/generating-an-ssh-key/

http://stackoverflow.com/questions/6565357/git-push-requires-username-and-password

1. In jupyter, open new terminal OR ssh into the instance via putty
1. git config --global --replace-all user.name {your github username}
1. git config --global --replace-all user.email {your email}
1. Below here, replace {rsafile} with /home/ubuntu/.ssh/id_rsa  This is your PRIVATE key.
1. cd /home/ubuntu/.ssh
1. ls
    - Does id_rsa already exist? If not, create by ssh-keygen -t rsa -b 4096 -N "" -C {your email} -f {rsa_file}
1. eval "$(ssh-agent -s)"
1. ssh-add {rsafile}
1. head {rsa_file}.pub - prints the PUBLIC key.
1. copy this public key to your github profile

# Make local clone of your repo

1. From github repo, click "clone or download".  Copy the *ssh* address.  Denote it {url}
1. cd /home/ubuntu/notebooks
1. git clone {url}
    - Should create subdirectory
    - Alternatively, create subdir manually.  Then append that dir to clone command.
1. git pull
1. test write privileges
    1. cd into subdir
    1. echo 'Hello World' >> testfile.md
    1. git add testfile.md
    1. git commit -m "Testing git write access"
    1. git push
    1. go to your github repo, refresh, see if changes appear

# OS commands in notebooks
You can issue OS commands within a jupyter notebook in several ways
- import os    then  os.system(commands).   See documentation on the python os module
- Lines starting with ! are sent as bash commands to the os
- Use the "magic" command %%bash to make an entire cell into a bash script

However, I usually find it easier to interact directly with the terminal.  As far as I can tell, none of these is interactive.  So if a command requires additional user input, it usually fails.  For example, you probably needed to respond "yes" to a message about rsa keys earlier.  